In [316]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle
import matplotlib.dates as mdates
import statistics
from fund.fund import Fund
from fund.states import States

In [317]:
market = Market()
save_parameters = False
fund = Fund("delta",States.BACKTEST)
fund.pull_portfolios()
fund.initialize_portfolios()

In [318]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [319]:
startDate = datetime(2016,1,1)
endDate = datetime(2023,1,1)
starting_capital = 100000
trades = fund.pull_trades()
portfolios = fund.pull_portfolio_states()

In [320]:
pt = pd.concat(portfolios)

In [321]:
pt["positions"] = [pickle.loads(x) for x in pt["positions"]]

In [322]:
t = pd.concat(trades)

In [323]:
non_params = ["date","sell_date","ticker","adjclose","sell_price","delta","projected_delta","position","holding",'close',
 'high',
 'low',
 'open',
 'volume',
 'adjhigh',
 'adjlow',
 'adjopen',
 'adjvolume',
 'divcash',
 'splitfactor',
 'year',
 'week',
 'quarter',
 'prediction',
"buy_price","amount","pv",
"swap"]
parameter_columns = [x for x in t.columns if x not in non_params]

In [324]:
t["holding"] = [(row[1]["sell_date"] - row[1]["date"]).days for row in t.iterrows()]

In [325]:
parameter_columns

['signal',
 'req',
 'value',
 'oc',
 'strategy',
 'portfolio_type',
 'modeler_type',
 'diversifier_type']

In [326]:
iterations = t.groupby(parameter_columns).mean().reset_index()[parameter_columns]

In [327]:
iterations

,signal,req,value,oc,strategy,portfolio_type,modeler_type,diversifier_type
0,0.05,0.05,False,False,speculation,universal,industry,basic
1,0.05,0.05,False,False,speculation,universal,industry,industry
2,0.05,0.05,False,False,speculation,universal,universal,basic
3,0.05,0.05,False,False,speculation,universal,universal,industry
4,0.05,0.05,True,False,speculation,universal,industry,basic
...,...,...,...,...,...,...,...,...
67,0.15,0.15,False,False,speculation,universal,universal,industry
68,0.15,0.15,True,False,speculation,universal,industry,basic
69,0.15,0.15,True,False,speculation,universal,industry,industry
70,0.15,0.15,True,False,speculation,universal,universal,basic


In [328]:
analysis = []
metric = "pv"
for row in iterations.iterrows():
    iteration_trades = t.copy()
    iteration_analysis = []
    for col in iterations.columns:
        param = row[1][col]
        iteration_trades = iteration_trades[(iteration_trades[col]==param)]
    positions = int(iteration_trades["position"].max())
    iteration_trades["delta"] = iteration_trades["delta"] + 1
    pivoted = iteration_trades.pivot_table(index="sell_date",columns="position",values="delta").fillna(1)
    for i in range(10):
        if i not in pivoted.columns:
            pivoted[i] = 1
    pivoted["pv"] = [sum([row[1][column] * 0.1 for column in range(10)]) for row in pivoted.iterrows()]
    pv = pivoted["pv"].cumprod().tail(1).item()
    current = row[1].to_dict()
    current["pv"] = pv
    analysis.append(current)

In [329]:
complete = pd.DataFrame(analysis)

In [330]:
complete.sort_values(metric,ascending=False).head(20)

,signal,req,value,oc,strategy,portfolio_type,modeler_type,diversifier_type,pv
19,0.05,0.15,False,False,speculation,universal,universal,industry,1.117104
11,0.05,0.10,False,False,speculation,universal,universal,industry,1.102365
41,0.10,0.15,False,False,speculation,universal,industry,industry,1.099333
33,0.10,0.10,False,False,speculation,universal,industry,industry,1.089743
18,0.05,0.15,False,False,speculation,universal,universal,basic,1.070168
3,0.05,0.05,False,False,speculation,universal,universal,industry,1.064028
25,0.10,0.05,False,False,speculation,universal,industry,industry,1.062372
10,0.05,0.10,False,False,speculation,universal,universal,basic,1.053026
17,0.05,0.15,False,False,speculation,universal,industry,industry,1.049069
40,0.10,0.15,False,False,speculation,universal,industry,basic,1.040428


In [331]:
results = []
strategy = "speculation"
for modeler_type in complete["modeler_type"].unique():
    for diversifier_type in complete["diversifier_type"].unique():
        results.append(complete[(complete["strategy"]==strategy) & (complete["modeler_type"]==modeler_type) & (complete["diversifier_type"]==diversifier_type)].sort_values("pv",ascending=False).head(1))
backtest_report = pd.concat(results)
best_it = backtest_report[(backtest_report["strategy"]==strategy)].copy()
backtest_report

,signal,req,value,oc,strategy,portfolio_type,modeler_type,diversifier_type,pv
40,0.10,0.15,False,False,speculation,universal,industry,basic,1.040428
41,0.10,0.15,False,False,speculation,universal,industry,industry,1.099333
18,0.05,0.15,False,False,speculation,universal,universal,basic,1.070168
19,0.05,0.15,False,False,speculation,universal,universal,industry,1.117104


In [332]:
sp5["GICS Sector"].unique()

array(['Industrials', 'Health Care', 'Information Technology',
       'Communication Services', 'Consumer Staples',
       'Consumer Discretionary', 'Utilities', 'Financials', 'Materials',
       'Real Estate', 'Energy'], dtype=object)

In [341]:
# all_portfolios = pt.copy()
# for modeler_type in complete["modeler_type"].unique():
#     for diversifier_type in complete["diversifier_type"].unique():
#         a_it = best_it[(best_it["modeler_type"]==modeler_type) & (best_it["diversifier_type"]==diversifier_type)]
#     iteration_trades = t.copy()
#     iteration_analysis = []
#     for col in a_it.columns:
#         if col != "pv":
#             param = row[1][col]
#             iteration_trades = iteration_trades[(iteration_trades[col]==param)]
#     positions = int(iteration_trades["position"].max())
#     iteration_trades["delta"] = iteration_trades["delta"] + 1
#     pivoted = iteration_trades.pivot_table(index="sell_date",columns="position",values="delta").fillna(1).reset_index()
#     for i in range(10):
#         if i not in pivoted.columns:
#             pivoted[i] = 1
#     pivoted["pv"] = [sum([row[1][column] * 0.1 for column in range(10)]) for row in pivoted.iterrows()]
#     pivoted["pv"] = pivoted[["pv"]].cumprod()
#     bench = pd.read_csv("./csv_files/FED/SPY.csv")
#     bench = p.column_date_processing(bench)
#     close_col = "adjclose"   
#     ds = bench[(bench["date"]<=endDate) &(bench["date"]>=startDate)].merge(pivoted[["sell_date","pv"]].rename(columns={"sell_date":"date"}),on="date",how="right").sort_values("date")
#     ds["bench_prog"] = (ds[close_col] - ds[close_col].iloc[0]) / ds[close_col].iloc[0] + 1
#     figure, axes = plt.subplots(figsize=(16, 8))
#     axes.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
#     axes.plot(ds["date"],
#              ds["bench_prog"])
#     axes.plot(ds["date"],
#              ds["pv"])
#     plt.title(f"{modeler_type}_{diversifier_type}_{positions}")
#     plt.show()

In [342]:
ds

,date,open,high,low,close,adjclose,volume,year,quarter,week,pv,bench_prog
0,2016-02-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.002628,NaN
1,2018-11-16,271.790009,274.750000,271.209991,273.730011,254.607559,126668000.0,2018.0,4.0,46.0,0.997139,NaN
2,2019-03-29,282.390015,282.839996,281.140015,282.480011,265.444702,82186800.0,2019.0,1.0,13.0,0.996996,NaN
3,2019-08-09,292.579987,293.239990,289.649994,291.619995,275.366333,93730000.0,2019.0,3.0,32.0,0.990401,NaN
4,2019-08-16,286.480011,289.329987,284.709991,288.850006,272.750671,83018300.0,2019.0,3.0,33.0,0.991068,NaN
5,2019-10-25,299.739990,302.200012,299.679993,301.600006,286.105255,45205400.0,2019.0,4.0,43.0,0.994822,NaN
6,2019-11-22,311.089996,311.239990,309.850006,310.959991,294.984314,44850200.0,2019.0,4.0,47.0,0.997006,NaN
7,2022-05-13,396.709991,403.179993,395.609985,401.720001,396.534882,104174400.0,2022.0,2.0,19.0,1.005549,NaN


In [335]:
industry_report = []
stock_report = []
all_trades = t.copy()
for modeler_type in complete["modeler_type"].unique():
    for diversifier_type in complete["diversifier_type"].unique():
        a_it = best_it[(best_it["modeler_type"]==modeler_type) & (best_it["diversifier_type"]==diversifier_type)]
        best_trades = all_trades.copy()
        for col in parameter_columns:
            best_trades = best_trades[best_trades[col]==a_it[col].item()]
        best_trades = best_trades[(best_trades["sell_date"]>=startDate) & (best_trades["sell_date"]<=endDate)]
        reporting = best_trades.merge(sp5.rename(columns={"Symbol":"ticker"}),on="ticker",how="left").sort_values("delta",ascending=False)[["date","sell_date","position","holding","projected_delta","value","delta","ticker","Security","GICS Sector","GICS Sub-Industry","modeler_type","diversifier_type","sell_price","buy_price"]]
        stock_report.append(reporting.sort_values("delta",ascending=False))
        reporting["delta"] = reporting["delta"] + 1
        reporting["year"] = [x.year for x in reporting["date"]]
        reporting["quarter"] = [x.quarter for x in reporting["date"]]
        reporting["modeler_type"] = modeler_type
        reporting["diversifier_type"] = diversifier_type
        ir = reporting.groupby(["year","GICS Sector","GICS Sub-Industry","modeler_type","diversifier_type"]).agg({"delta":np.prod,"buy_price":"mean","projected_delta":"mean","holding":"mean"}).sort_values("delta",ascending=False)
        ir["strategy"] = strategy
        industry_report.append(ir)    

In [336]:
stock_trades = pd.concat(stock_report).sort_values(["modeler_type","diversifier_type","date"],ascending=True)
stock_trades[stock_trades["diversifier_type"]=="industry"].head(10)

,date,sell_date,position,holding,projected_delta,value,delta,ticker,Security,GICS Sector,GICS Sub-Industry,modeler_type,diversifier_type,sell_price,buy_price
0,2016-06-08,2016-06-10,0,2,0.115131,False,-0.017065,ABMD,Abiomed,Health Care,Health Care Equipment,industry,industry,102.53,104.31
1,2016-07-06,2016-07-08,0,2,0.130104,False,0.039334,ABMD,Abiomed,Health Care,Health Care Equipment,industry,industry,114.15,109.83
2,2016-07-19,2016-07-22,0,3,0.159824,False,-0.010386,ABMD,Abiomed,Health Care,Health Care Equipment,industry,industry,116.25,117.47
3,2016-07-26,2016-07-29,0,3,0.141997,False,0.029227,ABMD,Abiomed,Health Care,Health Care Equipment,industry,industry,117.97,114.62
4,2016-08-02,2016-08-05,0,3,0.182235,False,0.006276,ABMD,Abiomed,Health Care,Health Care Equipment,industry,industry,120.25,119.50
5,2016-08-23,2016-08-26,0,3,0.183536,False,-0.015828,ABMD,Abiomed,Health Care,Health Care Equipment,industry,industry,118.76,120.67
6,2016-08-30,2016-09-02,0,3,0.184394,False,-0.027173,ABMD,Abiomed,Health Care,Health Care Equipment,industry,industry,117.43,120.71
7,2016-09-21,2016-09-23,0,2,0.215735,False,-0.012550,ABMD,Abiomed,Health Care,Health Care Equipment,industry,industry,123.53,125.10
8,2016-09-27,2016-09-30,0,3,0.211995,False,0.014678,ABMD,Abiomed,Health Care,Health Care Equipment,industry,industry,128.58,126.72
9,2016-10-04,2016-10-07,0,3,0.208916,False,0.005243,ABMD,Abiomed,Health Care,Health Care Equipment,industry,industry,128.47,127.80


In [337]:
stock_trades = pd.concat(stock_report).sort_values(["modeler_type","diversifier_type","holding"],ascending=False)
stock_trades.head(10)

,date,sell_date,position,holding,projected_delta,value,delta,ticker,Security,GICS Sector,GICS Sub-Industry,modeler_type,diversifier_type,sell_price,buy_price
107,2020-12-22,2021-01-08,0,17,0.058249,False,0.095500,ABMD,Abiomed,Health Care,Health Care Equipment,universal,industry,324.0600,295.81
84,2020-04-07,2020-04-17,0,10,0.069079,False,0.078275,ADBE,Adobe Inc.,Information Technology,Application Software,universal,industry,344.1100,319.13
129,2021-12-21,2021-12-31,0,10,0.052941,False,0.031450,ADBE,Adobe Inc.,Information Technology,Application Software,universal,industry,567.0600,549.77
36,2018-03-27,2018-04-06,0,10,0.210781,False,-0.031077,ABMD,Abiomed,Health Care,Health Care Equipment,universal,industry,286.2200,295.40
1,2016-03-23,2016-04-01,0,9,0.052658,False,0.032941,ABMD,Abiomed,Health Care,Health Care Equipment,universal,industry,96.5800,93.50
78,2019-10-29,2019-11-01,0,3,0.051627,False,0.150000,ABMD,Abiomed,Health Care,Health Care Equipment,universal,industry,211.4965,183.91
31,2018-02-13,2018-02-16,0,3,0.081176,False,0.145559,ABMD,Abiomed,Health Care,Health Care Equipment,universal,industry,267.7400,233.72
89,2020-06-02,2020-06-05,0,3,0.054222,False,0.104670,ABMD,Abiomed,Health Care,Health Care Equipment,universal,industry,249.8100,226.14
95,2020-08-25,2020-08-28,0,3,0.146981,False,0.084275,ADBE,Adobe Inc.,Information Technology,Application Software,universal,industry,516.4400,476.30
41,2018-05-08,2018-05-11,0,3,0.293478,False,0.072244,ABMD,Abiomed,Health Care,Health Care Equipment,universal,industry,372.6800,347.57


In [338]:
stock_trades = pd.concat(stock_report).sort_values(["modeler_type","diversifier_type","delta"],ascending=True)
stock_trades.head(10)

,date,sell_date,position,holding,projected_delta,value,delta,ticker,Security,GICS Sector,GICS Sub-Industry,modeler_type,diversifier_type,sell_price,buy_price
17,2016-10-25,2016-10-28,0,3,0.200558,False,-0.163044,ABMD,Abiomed,Health Care,Health Care Equipment,industry,basic,105.130000,125.610000
133,2018-07-24,2018-07-27,0,3,0.344605,False,-0.143258,ABMD,Abiomed,Health Care,Health Care Equipment,industry,basic,369.170000,430.900000
250,2021-11-30,2021-12-03,0,3,0.222351,False,-0.103216,ADBE,Adobe Inc.,Information Technology,Application Software,industry,basic,616.530000,687.490000
159,2018-10-23,2018-10-26,0,3,0.269127,False,-0.097897,ABMD,Abiomed,Health Care,Health Care Equipment,industry,basic,330.350000,366.200000
253,2021-12-15,2021-12-17,1,2,0.142454,False,-0.094688,ADBE,Adobe Inc.,Information Technology,Application Software,industry,basic,556.640000,614.860000
212,2020-09-02,2020-09-04,1,2,0.282051,False,-0.068207,ADBE,Adobe Inc.,Information Technology,Application Software,industry,basic,491.940000,527.950000
274,2022-09-20,2022-09-23,0,3,0.192528,False,-0.065041,ADM,ADM,Consumer Staples,Agricultural Products,industry,basic,81.305957,86.962024
265,2022-06-15,2022-06-17,1,2,0.138660,False,-0.064836,ADM,ADM,Consumer Staples,Agricultural Products,industry,basic,76.630212,81.943082
157,2018-10-17,2018-10-19,1,2,0.301871,False,-0.063224,ABMD,Abiomed,Health Care,Health Care Equipment,industry,basic,362.420000,386.880000
87,2017-11-29,2017-12-01,3,2,0.255765,False,-0.060785,ATVI,Activision Blizzard,Communication Services,Interactive Home Entertainment,industry,basic,60.100691,63.990358


In [339]:
pd.concat(industry_report).sort_values("delta",ascending=False).reset_index().head(20)

,year,GICS Sector,GICS Sub-Industry,modeler_type,diversifier_type,delta,buy_price,projected_delta,holding,strategy
0,2018,Health Care,Health Care Equipment,universal,industry,1.769192,340.131579,0.247036,3.078947,speculation
1,2018,Health Care,Health Care Equipment,universal,basic,1.458176,342.181556,0.250103,2.888889,speculation
2,2017,Information Technology,Application Software,industry,basic,1.385319,151.406471,0.190098,3.029412,speculation
3,2017,Information Technology,Application Software,industry,industry,1.327320,149.716667,0.185028,2.541667,speculation
4,2021,Information Technology,Application Software,industry,industry,1.288804,615.431765,0.162736,2.176471,speculation
5,2020,Information Technology,Application Software,universal,industry,1.260753,450.304706,0.124382,3.294118,speculation
6,2017,Health Care,Health Care Equipment,industry,basic,1.203106,165.457667,0.216239,2.633333,speculation
7,2019,Health Care,Health Care Equipment,universal,industry,1.186758,229.383333,0.083539,3.000000,speculation
8,2019,Health Care,Health Care Equipment,universal,basic,1.186758,229.383333,0.083539,3.000000,speculation
9,2020,Health Care,Health Care Equipment,industry,basic,1.165074,258.030000,0.119003,7.000000,speculation


In [340]:
if save_parameters:
    for portfolio in fund.portfolios:
        portfolio_type = portfolio["portfolio_type"]
        modeler_type = portfolio["modeler_type"]
        diversifier_type = portfolio["diversifier_type"]
        strategy = portfolio["portfolio_class"].strat_class.name
        optimal = complete[(complete["strategy"]==strategy) & (complete["modeler_type"]==modeler_type) & (complete["diversifier_type"]==diversifier_type)].sort_values("pv",ascending=False).head(1)
        strat_db = ADatabase(f"universal_financial_{modeler_type}_{diversifier_type}_{fund.state.value}")
        strat_db.connect()
        strat_db.drop("optimal_parameters")
        strat_db.store("optimal_parameters",optimal[parameter_columns])
        strat_db.disconnect()